In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
from skimage import io, color, util, exposure, filters, morphology
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from skimage.transform import resize
import shutil
import cv2
from skimage.filters import meijering, sato, frangi, hessian
from skimage.filters.rank import entropy
from skimage.morphology import disk, square
import cv2
import numpy as np
from scipy.ndimage import label
import matplotlib.pyplot as plt 

In [ ]:
# 工具函数
def batch_processing_for_folder(func, relative_path, samples_id, root):
    for i, sample_id in enumerate(samples_id):
        total_num = samples_id.count()
        # 打印进度
        print(round(i*100/total_num, 1), flush=True)
        root = Path(root)
        path = root / sample_id / relative_path
        func(path, relative_path, sample_id, root)

def remove_small_objects(binary_image, min_size=20):
    # Label connected components in the binary image
    labeled_image, num_objects = label(binary_image)
    # Iterate through each object and remove objects smaller than min_size
    for obj_id in range(1, num_objects + 1):
        obj_pixels = np.sum(labeled_image == obj_id)
        if obj_pixels < min_size:
            binary_image[labeled_image == obj_id] = 0
    return binary_image

def normalize_image(img_mask, img_have_masked, target_value=128):
    index = target_value / (img_have_masked.sum()/img_mask.sum())
    normalized_image = index * img_have_masked
    return normalized_image, index

# unuse
def draw_mask(shape):
    zero = np.zeros(shape)
    up_mask = zero.copy()
    up_mask_vertices = np.array([(0, 0), (shape[0]/2, shape[1]/2), (shape[0], 0)], dtype=np.int32)

    cv2.drawContours(up_mask, [up_mask_vertices], 0, color=(255, 255, 255), thickness=-1)
    
    down_mask = zero.copy()
    down_mask_vertices = np.array([(0, shape[1]), (shape[0]/2, shape[1]/2), (shape[0], shape[1])], dtype=np.int32)
    cv2.drawContours(down_mask, [down_mask_vertices], 0, color=(255, 255, 255), thickness=-1)
    
    left_mask = zero.copy()
    left_mask_vertices = np.array([(0, 0), (shape[0]/2, shape[1]/2), (0, shape[1])], dtype=np.int32)
    cv2.drawContours(left_mask, [left_mask_vertices], 0, color=(255, 255, 255), thickness=-1)
    
    right_mask = zero.copy()
    right_mask_vertices = np.array([(shape[0], 0), (shape[0]/2, shape[1]/2), (shape[0], shape[1])], dtype=np.int32)
    cv2.drawContours(right_mask, [right_mask_vertices], 0, color=(255, 255, 255), thickness=-1)
    
    return up_mask.astype(bool), down_mask.astype(bool), left_mask.astype(bool), right_mask.astype(bool)

In [ ]:
# select macula region
CROP_SIZE = 300

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Mouse clicked at ({}, {})".format(x, y))
        center = (x, y)
        global click_x, click_y
        click_x, click_y = x, y

root = PATH
samples_id = get_samples_id(root).index

for idx, sample_id in enumerate(samples_id):
    print(idx, sample_id)
    path_file = root / sample_id / 'median/aligned'
    list_imgs = list(path_file.iterdir())
    list_imgs.sort(key=lambda x: x.name.split('_')[2])
    
    # 按照early late middle 的顺序
    img_early, img_late, img_middle = (cv2.imread(str(list_imgs[0]), cv2.IMREAD_GRAYSCALE),
                                       cv2.imread(str(list_imgs[1]), cv2.IMREAD_GRAYSCALE),
                                       cv2.imread(str(list_imgs[2]), cv2.IMREAD_GRAYSCALE),)
    cv2.imshow('img_early', img_early)
    cv2.setMouseCallback('img_early', mouse_callback)
    
    while True:
        key = cv2.waitKey(0)
        if key == 13: # 13 is the ASCII code for the Enter key
            # Crop the image around the click location
            start_x = click_x - CROP_SIZE // 2
            start_y = click_y - CROP_SIZE // 2
            end_x = start_x + CROP_SIZE
            end_y = start_y + CROP_SIZE
            
            macula_early = img_early[start_y:end_y, start_x:end_x]
            macula_middle = img_middle[start_y:end_y, start_x:end_x]
            macula_late = img_late[start_y:end_y, start_x:end_x]

            # Display the cropped image
            cv2.imshow('Cropped img_early', macula_early)
            cv2.imshow('Cropped img_middle', macula_middle)
            cv2.imshow('Cropped img_late', macula_late)
            
            output_path = root / sample_id / 'roi'
            
            key = cv2.waitKey(0)
            if key == ord('r') or key == ord('R'):
                cv2.destroyWindow('Cropped img_early')
                cv2.destroyWindow('Cropped img_middle')
                cv2.destroyWindow('Cropped img_late')
                continue
            elif key == 13:
            
                if not output_path.exists():
                    output_path.mkdir()

                io.imsave(output_path/('01_' + list_imgs[0].name.split('_')[2] + '.png'), macula_early)
                io.imsave(output_path/('02_' + list_imgs[2].name.split('_')[2] + '.png'), macula_middle)
                io.imsave(output_path/('03_' + list_imgs[1].name.split('_')[2] + '.png'), macula_late)
                break
    
    # Wait for a key press and then exit
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# process images and save results
def img_processing(path, relative_path, sample_id, root, save_processed=False):
    global img_early_sato_1, img_early_sato_2, img_early_sato_1_5
    global img_vessel_1_5, img_vessel_1, img_vessel_2, img_vessel, img_vessel_ori
    list_imgs = list(path.iterdir())
    list_imgs.sort(key=lambda x: x.name)

    img_early_macula = io.imread(list_imgs[0])
    img_middle_macula = io.imread(list_imgs[1])
    img_late_macula = io.imread(list_imgs[2])
    
    img_early_clahe = exposure.equalize_adapthist(img_early_macula, clip_limit=0.01)
    img_early_sato_1 = sato(img_early_clahe, sigmas=2, mode='reflect', black_ridges=False)
    img_vessel_1 = img_early_sato_1 > filters.threshold_otsu(img_early_sato_1)

    img_early_sato_1_5 = sato(img_early_clahe, sigmas=1.5, mode='reflect', black_ridges=False)
    img_vessel_1_5 = img_early_sato_1_5 > filters.threshold_otsu(img_early_sato_1_5)

    img_early_sato_2 = sato(img_early_clahe, sigmas=1, mode='reflect', black_ridges=False)
    img_vessel_2 = img_early_sato_2 > filters.threshold_otsu(img_early_sato_2)

    img_vessel = util.invert(util.invert(img_vessel_1) * util.invert(img_vessel_2) * util.invert(img_vessel_1_5))
    img_vessel_ori = util.invert(util.invert(img_vessel_1) * util.invert(img_vessel_2) * util.invert(img_vessel_1_5))
    remove_small_objects(img_vessel)
    img_not_vessel = util.invert(img_vessel)
    
    img_early_without_vas = np.where(img_vessel, 0, img_early_macula)
    img_middle_without_vas = np.where(img_vessel, 0, img_middle_macula)
    img_late_without_vas = np.where(img_vessel, 0, img_late_macula)
    
    img_early_vas = np.where(img_vessel, img_early_macula, 0)
    img_middle_vas = np.where(img_vessel, img_middle_macula, 0)
    img_late_vas = np.where(img_vessel, img_late_macula, 0)
    
    norm_img_early_vas, early_index = normalize_image(img_vessel[30:270, 30:270], img_early_vas[30:270, 30:270])
    norm_img_middle_vas, middle_index = normalize_image(img_vessel[30:270, 30:270], img_middle_vas[30:270, 30:270])
    norm_img_late_vas, late_index = normalize_image(img_vessel[30:270, 30:270], img_late_vas[30:270, 30:270])
    
    norm_img_early_without_vas = img_early_without_vas[30:270, 30:270] * early_index
    norm_img_middle_without_vas = img_middle_without_vas[30:270, 30:270] * middle_index
    norm_img_late_without_vas = img_late_without_vas[30:270, 30:270] * late_index
    
    img_early_without_vas_clahe = exposure.equalize_adapthist(img_early_without_vas, clip_limit=0.01)

    output_path = path.parent.parent / 'ffa_processed'

    if not output_path.exists():
        output_path.mkdir()

    img_early_middle_vas = np.square((img_early_vas[30:270, 30:270]-img_middle_vas[30:270, 30:270]).astype(float))
    img_middle_late_vas = np.square((img_middle_vas[30:270, 30:270]-img_late_vas[30:270, 30:270]).astype(float))
    img_early_late_vas = np.square((img_early_vas[30:270, 30:270]-img_late_vas[30:270, 30:270]).astype(float))
    
    img_norm_early_middle_vas = np.square((norm_img_early_vas-norm_img_middle_vas).astype(float))
    img_norm_middle_late_vas = np.square((norm_img_middle_vas-norm_img_late_vas).astype(float))
    img_norm_early_late_vas = np.square((norm_img_early_vas-norm_img_late_vas).astype(float))
    
    img_early_middle = np.square((img_early_without_vas[30:270, 30:270]-img_middle_without_vas[30:270, 30:270]).astype(float))
    img_middle_late = np.square((img_middle_without_vas[30:270, 30:270]-img_late_without_vas[30:270, 30:270]).astype(float))
    img_early_late = np.square((img_early_without_vas[30:270, 30:270]-img_late_without_vas[30:270, 30:270]).astype(float))
    
    img_norm_early_middle = np.square((norm_img_early_without_vas-norm_img_middle_without_vas).astype(float))
    img_norm_middle_late = np.square((norm_img_middle_without_vas-norm_img_late_without_vas).astype(float))
    img_norm_early_late = np.square((norm_img_early_without_vas-norm_img_late_without_vas).astype(float))

    img_early_middle_all = np.square((img_early_macula[30:270, 30:270]-img_middle_macula[30:270, 30:270]).astype(float))
    img_middle_late_all = np.square((img_middle_macula[30:270, 30:270]-img_late_macula[30:270, 30:270]).astype(float))
    img_early_late_all = np.square((img_early_macula[30:270, 30:270]-img_late_macula[30:270, 30:270]).astype(float))
    
    img_norm_early_middle_all = np.square((img_early_macula[30:270, 30:270]*early_index-img_middle_macula[30:270, 30:270]*middle_index).astype(float))
    img_norm_middle_late_all = np.square((img_middle_macula[30:270, 30:270]*middle_index-img_late_macula[30:270, 30:270]*late_index).astype(float))
    img_norm_early_late_all = np.square((img_early_macula[30:270, 30:270]*early_index-img_late_macula[30:270, 30:270]*late_index).astype(float))
    
    if save_processed == True:
        io.imsave(output_path/'img_early_clahe.png', img_early_clahe)

        io.imsave(output_path/'src1_early.png', img_early_macula[30:270, 30:270])
        io.imsave(output_path/'src2_middle.png', img_middle_macula[30:270, 30:270])
        io.imsave(output_path/'src3_late.png', img_late_macula[30:270, 30:270])

        io.imsave(output_path/'vas1_early_middle.png', img_early_middle_vas)
        io.imsave(output_path/'vas2_middle_late.png', img_middle_late_vas)
        io.imsave(output_path/'vas3_early_late.png', img_early_late_vas)

        io.imsave(output_path/'norm_vas1_early_middle.png', img_norm_early_middle_vas)
        io.imsave(output_path/'norm_vas2_middle_late.png', img_norm_middle_late_vas)
        io.imsave(output_path/'norm_vas3_early_late.png', img_norm_early_late_vas)

        io.imsave(output_path/'cap1_early_middle.png', img_early_middle)
        io.imsave(output_path/'cap2_middle_late.png', img_middle_late)
        io.imsave(output_path/'cap3_early_late.png', img_early_late)

        io.imsave(output_path/'norm_cap1_early_middle.png', img_norm_early_middle)
        io.imsave(output_path/'norm_cap2_middle_late.png', img_norm_middle_late)
        io.imsave(output_path/'norm_cap3_early_late.png', img_norm_early_late)

        io.imsave(output_path/'all1_early_middle.png', img_early_middle_all)
        io.imsave(output_path/'all2_middle_late_all.png', img_middle_late_all)
        io.imsave(output_path/'all3_early_late_all.png', img_early_late_all)

        io.imsave(output_path/'norm_all1_early_middle.png', img_norm_early_middle_all)
        io.imsave(output_path/'norm_all2_middle_late.png', img_norm_middle_late_all)
        io.imsave(output_path/'norm_all3_early_late.png', img_norm_early_late_all)
    
    # 大血管
    df.loc[sample_id, 'early_middle_vas'] = np.sqrt(np.square((img_early_vas[30:270, 30:270]-img_middle_vas[30:270, 30:270]).astype(float)).sum() / (img_vessel[30:270, 30:270]).sum())
    df.loc[sample_id, 'middle_late_vas'] = np.sqrt(np.square((img_middle_vas[30:270, 30:270]-img_late_vas[30:270, 30:270]).astype(float)).sum() / (img_vessel[30:270, 30:270]).sum())
    df.loc[sample_id, 'early_late_vas'] = np.sqrt(np.square((img_early_vas[30:270, 30:270]-img_late_vas[30:270, 30:270]).astype(float)).sum() / (img_vessel[30:270, 30:270]).sum())
    
    
    df.loc[sample_id, 'norm_early_middle_vas'] = np.sqrt(np.square((norm_img_early_vas-norm_img_middle_vas).astype(float)).sum() / (img_vessel[30:270, 30:270]).sum())
    df.loc[sample_id, 'norm_middle_late_vas'] = np.sqrt(np.square((norm_img_middle_vas-norm_img_late_vas).astype(float)).sum() / (img_vessel[30:270, 30:270]).sum())
    df.loc[sample_id, 'norm_early_late_vas'] = np.sqrt(np.square((norm_img_early_vas-norm_img_late_vas).astype(float)).sum() / (img_vessel[30:270, 30:270]).sum())
    
    # 毛细血管
    df.loc[sample_id, 'early_middle'] = np.sqrt(np.square((img_early_without_vas[30:270, 30:270]-img_middle_without_vas[30:270, 30:270]).astype(float)).sum() / (util.invert(img_vessel[30:270, 30:270])).sum())
    df.loc[sample_id, 'middle_late'] = np.sqrt(np.square((img_middle_without_vas[30:270, 30:270]-img_late_without_vas[30:270, 30:270]).astype(float)).sum() / (util.invert(img_vessel[30:270, 30:270])).sum())
    df.loc[sample_id, 'early_late'] = np.sqrt(np.square((img_early_without_vas[30:270, 30:270]-img_late_without_vas[30:270, 30:270]).astype(float)).sum() / (util.invert(img_vessel[30:270, 30:270])).sum())
    
    df.loc[sample_id, 'norm_early_middle'] = np.sqrt(np.square((norm_img_early_without_vas-norm_img_middle_without_vas).astype(float)).sum() / (util.invert(img_vessel[30:270, 30:270])).sum())
    df.loc[sample_id, 'norm_middle_late'] = np.sqrt(np.square((norm_img_middle_without_vas-norm_img_late_without_vas).astype(float)).sum() / (util.invert(img_vessel[30:270, 30:270])).sum()) 
    df.loc[sample_id, 'norm_early_late'] = np.sqrt(np.square((norm_img_early_without_vas-norm_img_late_without_vas).astype(float)).sum() / (util.invert(img_vessel[30:270, 30:270])).sum())
    
    # all
    df.loc[sample_id, 'early_middle_all'] = np.sqrt(np.square((img_early_macula[30:270, 30:270]-img_middle_macula[30:270, 30:270]).astype(float)).sum() / (240*240))
    df.loc[sample_id, 'middle_late_all'] = np.sqrt(np.square((img_middle_macula[30:270, 30:270]-img_late_macula[30:270, 30:270]).astype(float)).sum() / (240*240))
    df.loc[sample_id, 'early_late_all'] = np.sqrt(np.square((img_early_macula[30:270, 30:270]-img_late_macula[30:270, 30:270]).astype(float)).sum() / (240*240))
    
    df.loc[sample_id, 'norm_early_middle_all'] = np.sqrt(np.square((img_early_macula[30:270, 30:270]*early_index-img_middle_macula[30:270, 30:270]*middle_index).astype(float)).sum() / (240*240))
    df.loc[sample_id, 'norm_middle_late_all'] = np.sqrt(np.square((img_middle_macula[30:270, 30:270]*middle_index-img_late_macula[30:270, 30:270]*late_index).astype(float)).sum() / (240*240)) 
    df.loc[sample_id, 'norm_early_late_all'] = np.sqrt(np.square((img_early_macula[30:270, 30:270]*early_index-img_late_macula[30:270, 30:270]*late_index).astype(float)).sum() / (240*240))
    
relative_path = 'ffa/roi'
root = PATH
samples_id = get_samples_id(root)

batch_processing_for_folder(img_processing, relative_path, samples_id, root)